# Preprocessing Digital Image Data

## Data Selection

### Loading and Resizing
+ Load images from the selected dataset
+ Resize images to a consistent dimension
+ Explain the rationale for choosing the specific dimensions
+ Discuss the trade-offs between image size and computational efficiency.

In [27]:
# =====================================================
# Imports & Config
# =====================================================
import os
import cv2
import random
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

ROOT_DIR = r"C:/Users/GEED/Documents/data-mining-l1/data/images/New Plant Diseases Dataset(Augmented)/train"

# =====================================================
# ImageDataset
# =====================================================
class ImageDataset:
    def __init__(self, root_dir, target_size=(224, 224),
                 use_augmentation=False, normalize=True,
                 subset_ratio=1.0, seed=42):
        self.root_dir = root_dir
        self.target_size = target_size
        self.use_augmentation = use_augmentation
        self.normalize = normalize

        if not os.path.isdir(root_dir):
            raise ValueError(f"Invalid directory: {root_dir}")

        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}
        self.idx_to_class = {}

        supported_ext = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')

        classes = sorted([
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d))
        ])

        if not classes:
            raise ValueError("No class subdirectories found")

        for idx, cls in enumerate(classes):
            self.class_to_idx[cls] = idx
            self.idx_to_class[idx] = cls

            cls_dir = os.path.join(root_dir, cls)
            for f in os.listdir(cls_dir):
                if f.lower().endswith(supported_ext):
                    self.image_paths.append(os.path.join(cls_dir, f))
                    self.labels.append(idx)

        if subset_ratio < 1.0:
            random.seed(seed)
            total = len(self.image_paths)
            subset_size = int(total * subset_ratio)

            indices = list(range(total))
            random.shuffle(indices)
            selected = indices[:subset_size]

            self.image_paths = [self.image_paths[i] for i in selected]
            self.labels = [self.labels[i] for i in selected]

            print(f"✓ Using subset: {subset_size}/{total} images ({subset_ratio*100:.0f}%)")

        print(f"✓ Dataset loaded: {len(self.image_paths)} images, {len(classes)} classes")

    def augment(self, image):
        if random.random() < 0.5:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)

        if random.random() < 0.5:
            image = image.rotate(random.uniform(-15, 15))

        if random.random() < 0.5:
            image = ImageEnhance.Brightness(image).enhance(random.uniform(0.8, 1.2))
            image = ImageEnhance.Contrast(image).enhance(random.uniform(0.8, 1.2))
            image = ImageEnhance.Color(image).enhance(random.uniform(0.8, 1.2))

        return image

    def load_image(self, idx):
        try:
            img_path = self.image_paths[idx]

            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, self.target_size)

            image = Image.fromarray(image)

            if self.use_augmentation:
                image = self.augment(image)

            image = np.asarray(image, dtype=np.float32) / 255.0

            if self.normalize:
                mean = np.array([0.485, 0.456, 0.406])
                std = np.array([0.229, 0.224, 0.225])
                image = (image - mean) / std

            image = np.transpose(image, (2, 0, 1))
            return image, self.labels[idx]

        except Exception as e:
            print(f"Error loading image: {e}")
            blank = np.zeros((3, *self.target_size), dtype=np.float32)
            return blank, self.labels[idx]


# =====================================================
# DataLoader
# =====================================================
class DataLoader:
    def __init__(self, dataset, batch_size=32, shuffle=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(dataset.image_paths))

    def __iter__(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

        for start in range(0, len(self.indices), self.batch_size):
            batch_idx = self.indices[start:start + self.batch_size]

            images, labels = [], []
            for idx in batch_idx:
                img, lbl = self.dataset.load_image(idx)
                images.append(img)
                labels.append(lbl)

            yield np.stack(images), np.array(labels)

    def __len__(self):
        return len(self.indices) // self.batch_size


In [28]:
# =====================================================
# Pipeline builder
# =====================================================
def build_image_pipeline(
    root_dir,
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    use_augmentation=False,
    normalize=True,
    subset_ratio=1.0,
    seed=42,
):
    dataset = ImageDataset(
        root_dir=root_dir,
        target_size=target_size,
        use_augmentation=use_augmentation,
        normalize=normalize,
        subset_ratio=subset_ratio,
        seed=seed
    )

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle
    )

    print(f"✓ Total images: {len(dataset.image_paths)}")
    print(f"✓ Classes: {len(dataset.class_to_idx)}")
    print(f"✓ Batches per epoch: {len(loader)}")

    return dataset, loader

# ---- Khởi tạo dataset & loader (chạy cell này là loader tồn tại)
dataset, loader = build_image_pipeline(
    root_dir=ROOT_DIR,
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    normalize=True
)


✓ Dataset loaded: 70295 images, 38 classes
✓ Total images: 70295
✓ Classes: 38
✓ Batches per epoch: 2196


In [29]:
# =====================================================
# Standalone resize & save
# =====================================================
OUTPUT_ROOT = r"C:/Users/GEED/Documents/data-mining-l1/data/images/subset_resized_224"
TARGET_SIZE = (224, 224)
NUM_WORKERS = min(16, os.cpu_count() * 2)

os.makedirs(OUTPUT_ROOT, exist_ok=True)

def resize_and_save(img_path, label):
    try:
        class_name = dataset.idx_to_class[label]
        out_dir = os.path.join(OUTPUT_ROOT, class_name)
        os.makedirs(out_dir, exist_ok=True)

        out_path = os.path.join(out_dir, os.path.basename(img_path))

        img = cv2.imread(img_path)
        if img is None:
            return False

        img = cv2.resize(img, TARGET_SIZE)
        return cv2.imwrite(out_path, img)
    except Exception:
        return False

tasks = list(zip(dataset.image_paths, dataset.labels))

success = 0
with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    futures = [
        executor.submit(resize_and_save, p, l)
        for p, l in tasks
    ]

    for f in tqdm(as_completed(futures), total=len(futures)):
        if f.result():
            success += 1

print(f"✅ Saved {success}/{len(tasks)} images")


100%|██████████| 70295/70295 [00:44<00:00, 1564.39it/s]


✅ Saved 70295/70295 images


In [ ]:
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.metrics import mutual_info_score
from scipy import ndimage
import time

def resize_image(img, size=(224,224), method=cv2.INTER_LINEAR):
    return cv2.resize(img, size, interpolation=method)

def measure_performance(func, *args):
    start = time.time()
    out = func(*args)
    elapsed = (time.time() - start) * 1000
    memory = out.nbytes / 1024
    return out, elapsed, memory



### Grayscale Conversion
+ Convert color images to grayscale where appropriate
+ Compare information retention between color and grayscale representations
+ Discuss when grayscale conversion is beneficial versus detrimental

In [30]:
# =====================================================
# Grayscale processing - FULL EPOCH
# Dataset: 70,295 images | 38 classes | 2,196 batches/epoch
# =====================================================

NUM_EPOCHS = 1  # đổi nếu cần

print(f"Dataset size      : {len(dataset.image_paths)} images")
print(f"Number of classes : {len(dataset.class_to_idx)}")
print(f"Batches / epoch   : {len(loader)}")

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")

    for batch_idx, (images, labels) in enumerate(tqdm(loader, total=len(loader))):
        gray_images = (
            0.299 * images[:, 0] +
            0.587 * images[:, 1] +
            0.114 * images[:, 2]
        )

        gray_images = gray_images[:, None, :, :]

        if batch_idx == 0:
            print(" RGB batch shape :", images.shape)
            print(" Gray batch shape:", gray_images.shape)

            print(" Gray batch shape:", gray_images.shape)


Dataset size      : 70295 images
Number of classes : 38
Batches / epoch   : 2196

Epoch 1/1


  0%|          | 1/2196 [00:00<04:42,  7.78it/s]

 RGB batch shape : (32, 3, 224, 224)
 Gray batch shape: (32, 1, 224, 224)
 Gray batch shape: (32, 1, 224, 224)


2197it [03:39, 10.00it/s]                          


In [ ]:
def rgb_to_gray_luminance(img):
    return 0.299*img[:,:,0] + 0.587*img[:,:,1] + 0.114*img[:,:,2]

def mutual_information(rgb, gray):
    rgb_flat = rgb.mean(axis=2).ravel()
    gray_flat = gray.ravel()
    return mutual_info_score(rgb_flat, gray_flat)

def information_retention(ssim_val, mi, mi_max):
    mi_norm = mi / mi_max
    return (ssim_val + mi_norm) / 2 * 100

def gray_variance(gray):
    return np.var(gray)


### Normalization
+ Normalization: Apply pixel normalization (e.g., scaling to [0,1] or [-1,1]) and implement standardization (zero mean, unit variance).
+ Compare different normalization techniques and their effects on the data distribution.


In [31]:
# =====================================================
# Grayscale Normalization - Batch-wise
# Input : gray_images (B, 1, H, W)
# Output: gray_norm   (B, 1, H, W)
# =====================================================

def normalize_gray_batchwise(gray, eps=1e-6):
    """
    Chuẩn hóa grayscale theo toàn batch (batch-wise normalization)

    Parameters
    ----------
    gray : np.ndarray
        Grayscale images, shape (B, 1, H, W)
    eps : float
        Hệ số nhỏ để tránh chia cho 0

    Returns
    -------
    gray_norm : np.ndarray
        Grayscale đã được chuẩn hóa
    """
    mean = gray.mean(axis=(0, 1, 2, 3), keepdims=True)
    std  = gray.std(axis=(0, 1, 2, 3), keepdims=True)
    return (gray - mean) / (std + eps)


# ---- Áp dụng normalization cho batch grayscale
gray_norm = normalize_gray_batchwise(gray_images)

# ---- DEBUG nhanh (chỉ kiểm tra batch đầu)
print("Gray shape        :", gray_images.shape)
print("Gray norm shape   :", gray_norm.shape)
print("Mean after norm   :", gray_norm.mean())
print("Std after norm    :", gray_norm.std())


Gray shape        : (23, 1, 224, 224)
Gray norm shape   : (23, 1, 224, 224)
Mean after norm   : -2.728764665671414e-17
Std after norm    : 0.9999988376733839


In [ ]:
def minmax_norm(img):
    return (img - img.min()) / (img.max() - img.min() + 1e-6)

def zscore_norm(img):
    return (img - img.mean()) / (img.std() + 1e-6)

def describe(img):
    return {
        "min": img.min(),
        "max": img.max(),
        "mean": img.mean(),
        "median": np.median(img),
        "std": img.std()
    }


### Edge Detection (Optional Bonus)
+ Apply edge detection algorithms (Sobel, Prewitt, Canny)
+ Extract edge features from images, and then visualize detected edges and discuss their significance for your
chosen dataset.

In [32]:
# =====================================================
# Edge Detection - Batch-wise (Unified Version)
# Input : gray_norm (B, 1, H, W)
# Output: edges     (B, H, W)
# =====================================================

def sobel_edges(gray_norm):
    """
    Sobel edge detection cho grayscale đã normalize batch-wise

    Parameters
    ----------
    gray_norm : np.ndarray
        Shape (B, 1, H, W)

    Returns
    -------
    edges : np.ndarray
        Shape (B, H, W)
    """
    gray = gray_norm[:, 0]  # (B, H, W)
    edges = []

    for g in gray:
        sx = ndimage.sobel(g, axis=0)
        sy = ndimage.sobel(g, axis=1)
        edges.append(np.hypot(sx, sy))

    return np.stack(edges)


def prewitt_edges(gray_norm):
    """
    Prewitt edge detection cho grayscale đã normalize batch-wise
    """
    gray = gray_norm[:, 0]
    edges = []

    for g in gray:
        px = ndimage.prewitt(g, axis=0)
        py = ndimage.prewitt(g, axis=1)
        edges.append(np.hypot(px, py))

    return np.stack(edges)


def canny_edges(gray_norm, low=50, high=150):
    """
    Canny edge detection (cần uint8 [0,255])
    """
    gray = gray_norm[:, 0]
    edges = []

    for g in gray:
        g_uint8 = cv2.normalize(
            g, None, 0, 255, cv2.NORM_MINMAX
        ).astype(np.uint8)

        edges.append(cv2.Canny(g_uint8, low, high))

    return np.stack(edges)


In [33]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_edges(gray, edges, idx=0, title="Edges"):
    """
    Visualize grayscale và edge map tương ứng

    Parameters
    ----------
    gray : np.ndarray
        (B, 1, H, W) hoặc (B, H, W)
    edges : np.ndarray
        (B, H, W)
    idx : int
        Index ảnh trong batch
    """

    # ---- đảm bảo đúng shape để vẽ
    if gray.ndim == 4:
        gray_img = gray[idx, 0]
    else:
        gray_img = gray[idx]

    edge_img = edges[idx]

    # ---- rescale để hiển thị
    gray_img = (gray_img - gray_img.min()) / (gray_img.ptp() + 1e-6)
    edge_img = (edge_img - edge_img.min()) / (edge_img.ptp() + 1e-6)

    plt.figure(figsize=(8, 4))

    plt.subplot(1, 2, 1)
    plt.imshow(gray_img, cmap="gray")
    plt.title("Grayscale")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(edge_img, cmap="gray")
    plt.title(title)
    plt.axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
def edge_coverage(edge):
    return np.count_nonzero(edge) / edge.size * 100

def mean_edge_intensity(edge):
    return edge[edge > 0].mean()

def edge_density_per_class(edges_by_class):
    return np.mean([edge_coverage(e) for e in edges_by_class])


Main Function for testing purpose and visualization

In [1]:
def main(
    loader,
    edge_method="sobel",
    visualize=True,
    vis_idx=0,
    max_batches=1
):
    """
    Main pipeline xử lý ảnh:
    RGB → Grayscale → Normalize → Edge detection → (Visualize)

    Parameters
    ----------
    loader : DataLoader
        DataLoader đã được build sẵn
    edge_method : str
        "sobel" | "prewitt" | "canny"
    visualize : bool
        Có hiển thị kết quả hay không
    vis_idx : int
        Index ảnh trong batch để visualize
    max_batches : int
        Số batch xử lý (debug nhanh, tránh chạy toàn bộ dataset)
    """

    print("🚀 Starting image processing pipeline")
    print(f"Edge method   : {edge_method}")
    print(f"Visualize     : {visualize}")
    print(f"Max batches   : {max_batches}")
    print("-" * 50)

    for batch_idx, (images, labels) in enumerate(loader):

        # ---- 1. RGB → Grayscale (luminance)
        gray = (
            0.299 * images[:, 0] +
            0.587 * images[:, 1] +
            0.114 * images[:, 2]
        )
        gray = gray[:, None, :, :]   # (B, 1, H, W)

        # ---- 2. Normalize grayscale (batch-wise)
        gray_norm = normalize_gray_batchwise(gray)

        # ---- 3. Chuẩn bị cho edge detection
        gray_for_edge = gray_norm[:, 0]   # (B, H, W)

        # ---- 4. Edge detection
        if edge_method == "sobel":
            edges = sobel_edges_batchwise(gray_for_edge)
            edge_title = "Sobel edges"

        elif edge_method == "prewitt":
            edges = prewitt_edges_batchwise(gray_for_edge)
            edge_title = "Prewitt edges"

        elif edge_method == "canny":
            edges = canny_edges_per_image(gray_for_edge)
            edge_title = "Canny edges"

        else:
            raise ValueError("edge_method must be 'sobel', 'prewitt', or 'canny'")

        # ---- 5. Visualization (optional)
        if visualize:
            visualize_edges(
                gray_norm,
                edges,
                idx=vis_idx,
                title=edge_title
            )

        # ---- 6. Thoát sớm nếu debug
        if batch_idx + 1 >= max_batches:
            break

    print("✅ Pipeline finished")


Image (RGB)
 → Resize
 → Normalize
 → Grayscale
 → Gray Normalization
 → Edge Detection
 → Visualization / Feature Extraction